In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install accelerate datasets rouge_score wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1

In [3]:
import os
import nltk
import random

import torch
import numpy as np

import datasets
import transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, BertTokenizerFast, EncoderDecoderModel

In [4]:
ENCODER_MAX_LENGTH = 256
DECODER_MAX_LENGTH = 16
SEED = 42
MODEL_NAME = "/content/drive/MyDrive/NLP/자연어처리 11조/bertshare_save/results"

In [5]:
from transformers import BertTokenizerFast, EncoderDecoderModel

# "kykim/bertshared-kor-base"
# 토크나이저 로드
tokenizer = BertTokenizerFast.from_pretrained(MODEL_NAME)

# EncoderDecoderModel 로드
model = EncoderDecoderModel.from_pretrained(MODEL_NAME)

# decoder_start_token_id 설정
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
#ValueError: Make sure to set the pad_token_id attribute of the model's configuration.

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [6]:
train_dir = "/content/drive/MyDrive/NLP/자연어처리 11조/train.json의 사본"
val_dir = "/content/drive/MyDrive/NLP/자연어처리 11조/test.json의 사본"

In [7]:
data = datasets.load_dataset("json", data_files={"train": train_dir, "test": val_dir})

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
def flatten(example):
  items = []
  for item in example['body']:
    participant_token = '<' + item['participantID'] + '>'
    items.append(participant_token)
    items.append(item['utterance'])
  response = items.pop()
  context = ' '.join(items)
  return {'context':context, 'response':response}

data = data.map(flatten, remove_columns=['body'])

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [9]:
train_data_txt = data['train']
val_data_txt = data['test']

In [ ]:
train_data_txt[3]

{'context': '<P01> 걔는 솔직히 쎄한거 첨부터 확 티안남? <P02> 근데 니는 어떻게 알았어 나는 이름랑 뭔가 잘 맞는거 같애서 <P01> 별로인거? 걔 하는짓보고 <P02> 둘이 잘사귀겠네 했는데 무슨짓했어 <P01>',
 'response': '애가 좀 과시하려는것도 있고 소유욕 쩔잖아 과시욕 소유욕'}

In [10]:
special_tokens_dict = {'additional_special_tokens': ['<P01>', '<P02>']}
tokenizer.add_special_tokens(special_tokens_dict)
model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))

Embedding(42002, 768, padding_idx=0)

In [ ]:
print(tokenizer.all_special_tokens)
print(tokenizer.all_special_ids)

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '<P01>', '<P02>']
[1, 3, 0, 2, 4, 42000, 42001]


In [ ]:
tokenizer.convert_tokens_to_ids('<P01>')

42000

In [11]:
### preprocess 및 tokenize
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["context"], batch["response"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, ENCODER_MAX_LENGTH, DECODER_MAX_LENGTH
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

val_data = val_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, ENCODER_MAX_LENGTH, DECODER_MAX_LENGTH
    ),
    batched=True,
    remove_columns=val_data_txt.column_names,
)

Map:   0%|          | 0/500000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [12]:
### 학습

nltk.download("punkt", quiet=True)

metric = datasets.load_metric("rouge")


def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

<ipython-input-12-546c7a01423d>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("rouge")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [16]:
### Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/자연어처리 11조/bertshare_save/results",
    overwrite_output_dir=True,
    save_strategy = "steps",
    save_total_limit = 3,
    save_steps = 1000,
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    #learning_rate=5e-05,
    warmup_steps=300,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [14]:
import wandb
#wandb.init(mode="disabled")

In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: 

In [17]:
checkpoint_path = "/content/drive/MyDrive/NLP/자연어처리 11조/bertshare_save/results/checkpoint-15000"
trainer.train(resume_from_checkpoint=True)

There were missing keys in the checkpoint model loaded: ['encoder.embeddings.word_embeddings.weight', 'encoder.embeddings.position_embeddings.weight', 'encoder.embeddings.token_type_embeddings.weight', 'encoder.embeddings.LayerNorm.weight', 'encoder.embeddings.LayerNorm.bias', 'encoder.encoder.layer.0.attention.self.query.weight', 'encoder.encoder.layer.0.attention.self.query.bias', 'encoder.encoder.layer.0.attention.self.key.weight', 'encoder.encoder.layer.0.attention.self.key.bias', 'encoder.encoder.layer.0.attention.self.value.weight', 'encoder.encoder.layer.0.attention.self.value.bias', 'encoder.encoder.layer.0.attention.output.dense.weight', 'encoder.encoder.layer.0.attention.output.dense.bias', 'encoder.encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.encoder.layer.0.intermediate.dense.weight', 'encoder.encoder.layer.0.intermediate.dense.bias', 'encoder.encoder.layer.0.output.dense.weight', 'encoder.encoder.la

Step,Training Loss
15100,5.006500
15200,5.018600
15300,5.020100
15400,5.038800
15500,5.010600
15600,5.002100
15700,4.904700
15800,4.894000
15900,4.868500
16000,4.869100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co

Step,Training Loss
15100,5.006500
15200,5.018600
15300,5.020100
15400,5.038800
15500,5.010600
15600,5.002100
15700,4.904700
15800,4.894000
15900,4.868500
16000,4.869100


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co

TrainOutput(global_step=39065, training_loss=2.9328572932269834, metrics={'train_runtime': 40087.3667, 'train_samples_per_second': 62.364, 'train_steps_per_second': 0.974, 'total_flos': 4.402466688e+17, 'train_loss': 2.9328572932269834, 'epoch': 5.0})

In [18]:
trainer.save_model("/content/drive/MyDrive/NLP/자연어처리 11조/bertshare_save/results")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'min_length': 8, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 1.5, 'no_repeat_ngram_size': 2}


In [ ]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:643: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


{'eval_loss': 5.2169575691223145,
 'eval_rouge1': 0.0679,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0661,
 'eval_rougeLsum': 0.067,
 'eval_gen_len': 10.479,
 'eval_runtime': 2459.8388,
 'eval_samples_per_second': 20.327,
 'eval_steps_per_second': 0.318,
 'epoch': 2.0}

In [32]:
def generate_response(test_samples, model):
    inputs = tokenizer(
        test_samples["context"],
        padding="max_length",
        truncation=True,
        max_length=ENCODER_MAX_LENGTH,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


#model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

test_samples = val_data_txt.select(range(3, 70, 5))

#summaries_before_tuning = generate_response(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_response(test_samples, model)[1]

In [20]:
from tabulate import tabulate

In [33]:
print(
    tabulate(
        zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            #summaries_before_tuning,
        ),
        headers=["Id", "response after", "response before"],
    )
)
print("\ntarget response:\n")
print(
    tabulate(list(enumerate(test_samples["response"])), headers=["Id", "target response"])
)
print("\nsource context:\n")
print(tabulate(list(enumerate(test_samples["context"])), headers=["Id", "context"]))

  Id  response after
----  ----------------------------------
   0  나두나두 보고팡 이모티콘
   1  난 밥먹엇어 시스템동영상
   2  그치그치 합격이지이
   3  10억은 한도보다 더 큰돈이지
   4  서울가는게 피곤하대
   5  헐 진짜?? 나도 집가고싶당
   6  그러게말이다 근데 그게 더 나으려나
   7  응응 이름이네도 할머니댁이지
   8  이름언니 독수리기타기타웃겨
   9  잘했오?? 잘잤어?
  10  오빠도 최고다 최고야
  11  갤9에 내폰으로 찍엇네
  12  난 집왔어 이름는??
  13  진짜?? 이모티콘흑흑

target response:

  Id  target response
----  ----------------------------------------------------------------------------------------------------------
   0  나두우  이모티콘
   1  오
   2  과연 이게 약간 누르면 튀어 나오는 승 질이 있어서 야발1 이럴 수도 있다애오  농담이고
   3  좋아좋아 나는 글 마저쓰고 쓰러져야겠다
   4  괜추
   5  아아??
   6  세상 불효자네 이런 불효자를 봤나
   7  몰랑할머니댁 간다하고 갔엉 아마 조금있으면 도착할껄?
   8  기타는 릠 맨날 길 걸어가면서 인사하쟈너
   9  헐 뭐야 이쫘식아 뭐야뭐야뭐야뭐얌 아프지마 이름  이모티콘 헐 이름 어떻게 이렇게 귀여운 이모티콘을 이모티콘
  10  비하인드짤 대방출 화가 잔뜩 났어 시스템사진 퀜처에서
  11  이모티콘 이모티콘
  12  난 지금 퇴근해
  13  진짜 일만으로도 힘든데

source context:

  Id  context
----  -------------------------------------------------------------------------------------------------